# Complex TFA multiple Inversions

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando.vis import mpl

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
from datetime import datetime
today = datetime.today()
# dd/mm/YY
d4 = today.strftime("%d-%b-%Y-%Hh:%Mm")

### Auxiliary functions

In [3]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [4]:
#choose the folder of the desired model
path = 'data/thick-inter-body/'

In [5]:
with open(path+'model.pickle') as w:
        model = pickle.load(w)

### Observation points and observed data

In [6]:
with open(path+'data.pickle') as w:
        data = pickle.load(w)

In [7]:
dobs = data['tfa_obs']
xp = data['grid'][0]
yp = data['grid'][1]
zp = data['grid'][2]
N = data['grid'][3]

In [8]:
zp

array([ -59.69,  -58.82,  -57.58, ..., -162.48, -168.91, -174.9 ])

### Parameters of the initial model

In [9]:
M = 20 # number of vertices per prism
L = 10 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']
decs = model['dec']

int_min = 10.
int_max = 15.
intensity = np.linspace(int_min, int_max, 6)

# depth to the top, thickness and radius
z0_min = 10.
z0_max = 210.
z0 = np.linspace(z0_min, z0_max, 6)
dz = 650.
r = 800.
x0 = -300.
y0 = 300.

# main field
inc, dec = data['main_field']

In [10]:
z0

array([ 10.,  50.,  90., 130., 170., 210.])

In [11]:
intensity

array([10., 11., 12., 13., 14., 15.])

### Limits

In [12]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -4000.
x0max = 4000.
y0min = -4000.
y0max = 4000.
dzmin = 200.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [13]:
# variation for derivatives
deltax = 0.01*np.max(100.)
deltay = 0.01*np.max(100.)
deltar = 0.01*np.max(100.)
deltaz = 0.01*np.max(100.)

### Outcropping parameters

In [14]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

In [15]:
# output of inversion
inversion = dict()

### Regularization parameters

In [16]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-4  # adjacent radial distances within each prism
a2 = 1.0e-5   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4    # vertically adjacent origins
a6 = 1.0e-7   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-7     # zero order Tikhonov on thickness of each prism

In [17]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [18]:
alpha

array([1.e-04, 1.e-05, 0.e+00, 0.e+00, 1.e-04, 1.e-07, 1.e-07])

In [19]:
foldername = '45477'

In [20]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [21]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [22]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = np.array([a1, a2, a3, a4, a5, a6, a7])
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]
inversion['intensity'] = intensity

### Inversion

In [23]:
inversion_results = []
for j, z in enumerate(z0):
    for k, i in enumerate(intensity):
        alpha = np.array([a1, a2, a3, a4, a5, a6, a7])
        inv = j*z0.size + k +1
        print 'inversion: %d  top: %d  intensity: %d' % (inv, z, i)
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.l1_levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz
        )
        inversion_results.append([m_est, phi_list, model_list, dobs - d_fit])

inversion: 1  top: 10  intensity: 10
it:  0   it_marq:  0   lambda: 1e+01   init obj.: 7.77236e+01  fin obj.: 7.42155e+01
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 7.42155e+01  fin obj.: 6.36837e+01
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 6.36837e+01  fin obj.: 4.74362e+01
it:  3   it_marq:  0   lambda: 1e-02   init obj.: 4.74362e+01  fin obj.: 4.27046e+01
it:  4   it_marq:  0   lambda: 1e-03   init obj.: 4.27046e+01  fin obj.: 3.92292e+01
it:  5   it_marq:  0   lambda: 1e-04   init obj.: 3.92292e+01  fin obj.: 3.72619e+01
it:  6   it_marq:  0   lambda: 1e-05   init obj.: 3.72619e+01  fin obj.: 3.74418e+01
it:  6   it_marq:  1   lambda: 1e-04   init obj.: 3.72619e+01  fin obj.: 3.74413e+01
it:  6   it_marq:  2   lambda: 1e-03   init obj.: 3.72619e+01  fin obj.: 3.74341e+01
it:  6   it_marq:  3   lambda: 1e-02   init obj.: 3.72619e+01  fin obj.: 3.73297e+01
it:  6   it_marq:  4   lambda: 1e-01   init obj.: 3.72619e+01  fin obj.: 3.69750e+01
it:  7   it_marq:  0   lambd

# Results

In [24]:
inversion['results'] = inversion_results

### Folder to save the results

In [25]:
if foldername == '':
    mypath = path+'l1_tfa_inversion/multiple-'+d4 #default folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)
else:
    mypath = path+'l1-tfa-inversion/multiple-36-'+foldername #defined folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

In [26]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)

In [27]:
mypath

'data/thick-inter-body/l1-tfa-inversion/multiple-36-45477'